In [28]:
import pandas as pd
import sys
import time
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import matplotlib.pyplot as plt
from MCLP import posiblelocations,n, costumerscoords, costumers, maximumdistance,facilities
from itertools import combinations


dist2 = euclidean_distances(posiblelocations, costumerscoords)

# Convert back to dataframe.
df = pd.DataFrame(dist2, columns=costumers.index, index=posiblelocations.index)

df['coverednodes'] = df[df <= maximumdistance].count(1)
df.sort_values(by=['coverednodes'], ascending=False, inplace=True)
# This does the sorting, by covered nodes(it maximizes the covered nodes)

dfheuristic = df.iloc[:facilities]

def outputText(selectedlocations, totalcoverednodes, indexofnodes,totalpopulationserved ):
        print("\nThe used locationes are :", selectedlocations)
        # print("\nThere are "+str(totalcoverednodes)+" nodes covered\n")
        print('\nThis are the covered nodes: ', indexofnodes)
        print("\nThe covered population is :", totalpopulationserved)

def binary(x,y):
    dfheuristic  = df.iloc[[x,y]]
    selectedlocations = list(dfheuristic.index.values)
    for i in range(facilities):
        for j in range(n):
            g = np.where(dfheuristic .iloc[i, j] >= maximumdistance, True, False)
            if g:
                dfheuristic.iloc[i, j] = 1
            else:
                dfheuristic.iloc[i, j] = 0

    dfheuristic.loc['Total',:] = dfheuristic.sum(axis=0)
    dfheuristic [dfheuristic.iloc[-1:] > 1 ] = 1
    Binary = dfheuristic.loc['Total',:].values.tolist()   
    Binary = Binary[:-1] 
    costumers['Binary'] =Binary

    totalcoverednodes = 0
    totalpopulationserved = 0
    for j in range(1,n):
        z = np.where(costumers.loc[j,'Binary'] == 1, True, False)
        if z:
            totalcoverednodes += 1
            totalpopulationserved += costumers.loc[j,'Demand']

    # Get the index of the node covered
    indexofnodes = costumers[costumers['Binary']== 1].index.values
    #outputText(selectedlocations, totalcoverednodes, indexofnodes,totalpopulationserved)
    return [selectedlocations, totalcoverednodes, indexofnodes,totalpopulationserved]



for x in range(posiblelocations["X"].count()):
    if x <= 1: continue
    value = binary(0,x) 
    print(value)

for x in range(posiblelocations["X"].count()):
    
    if x <= 1: continue
    binary(x,1)      



[[1, 3], 33, array([ 1,  3,  4,  5,  6,  7,  8, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21,
       23, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 44, 45, 47, 49, 50]), 1068]
[[1, 2], 37, array([ 1,  3,  4,  5,  6,  7,  8, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21,
       22, 23, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 44,
       45, 47, 49, 50]), 1211]
[[1, 5], 31, array([ 1,  2,  4,  5,  6,  7,  8, 10, 13, 15, 16, 17, 18, 19, 20, 21, 23,
       25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 39, 44, 45, 50]), 1029]


/tmp/ipykernel_1103/2750907331.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfheuristic.loc['Total',:] = dfheuristic.sum(axis=0)
/tmp/ipykernel_1103/2750907331.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfheuristic.loc['Total',:] = dfheuristic.sum(axis=0)
/tmp/ipykernel_1103/2750907331.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfheuristic.loc['Total',:] = dfheuristic.sum(axis=0)
/tmp/ipykernel_1103/27509073